In [1]:
import cv2
import mediapipe as mp
import time
import math
import matplotlib.pyplot as plt

In [2]:
class HandAnchorCalculation:  

    def __init__(self, mode = False, maxhands = 2, detectionConf = 0.69, trackingConf = 0.69):
        self.mode = mode
        self.maxhands = maxhands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        self.mphands = mp.solutions.hands
        self.hands = self.mphands.Hands(static_image_mode = self.mode, 
                                        max_num_hands = self.maxhands,
                                        min_detection_confidence = self.detectionConf,
                                        min_tracking_confidence = self.trackingConf)
        self.mpDraw = mp.solutions.drawing_utils
        
        self.tipIds = [4, 8, 12, 16, 20]
    
    
    def findHands(self, img, draw = True, fliptype = True):
        imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgrgb)
        allhands = []
        h, w, c = img.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                handlist = {}
                lmlist = []
                xlist = []
                ylist = []
                
                for id, lm in enumerate(handLms.landmark):
                    px, py = int(lm.x * w), int(lm.y * h)
                    lmlist.append([px, py])
                    xlist.append(px)
                    ylist.append(py)
                
                # define hand box boundary
                xmin, xmax, ymin, ymax = min(xlist), max(xlist), min(ylist), max(ylist)
                bboxw, bboxh = xmax - xmin, ymax - ymin
                bbox = xmin, ymin, bboxw, bboxh
                cx, cy = xmin + (bboxw // 2), ymin + (bboxh // 2)
                
                # define the hand zone attribute
                handlist["lmlist"] = lmlist
                handlist["bbox"] = bbox
                handlist["center"] = (cx, cy)                
                if (fliptype == True):
                    if (handType.classification[0].label == "Right"):
                        handlist["type"] = "Left"
                    else:
                        handlist["type"] = "Right"
                else:
                    handlist["type"] = handType.classification[0].label
                    
                allhands.append(handlist)
                
                # draw the hand zone on frame
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mphands.HAND_CONNECTIONS)
                    cv2.rectangle(img, (xmin-10, ymin-10), (xmax+10, ymax+10), (255, 0, 255), 2)
                    cv2.putText(img, handlist["type"], (xmin - 15, ymin - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
        
        if draw:
            return allhands, img
        else:
            return allhands
    
    
    def findFingersUp(self, handlist, img):
        fingers = []
        handtype = handlist["type"]
        lmlist = handlist["lmlist"]
        bbox = handlist["bbox"]
        
        if (self.results.multi_hand_landmarks):
            # Thumb
            if handtype == "Right":
                if lmlist[self.tipIds[0]][0] > lmlist[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if lmlist[self.tipIds[0]][0] < lmlist[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            # 4 Fingers
            for id in range(1, 5):
                if lmlist[self.tipIds[id]][1] < lmlist[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)                    
            
            cv2.putText(img, str(fingers) + "  " + str(sum(fingers)), (bbox[0] - 15, bbox[1] + bbox[3] + 35), cv2.FONT_HERSHEY_PLAIN, 1, (0, 100, 255), 1)
            
        return fingers
    
    def findHandDistance(self, p1, p2, img=None):
        x1, y1 = p1
        x2, y2 = p2
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        length = round(math.hypot(x2 - x1, y2 - y1), 2)
        info = (x1, y1, x2, y2, cx, cy)
        if img is not None:
            cv2.circle(img, (x1, y1), 9, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 9, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 9, (255, 0, 255), cv2.FILLED)
            
            cv2.putText(img, str(length), (cx, cy), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            return length,info, img
        else:
            return length, info
        
        
    def update_pts(self, params, x, y):
        global x_init, y_init, butt
        
        params["side"] = butt
        params["top_left_pt"] = (min(x_init, x), min(y_init, y))
        params["bottom_right_pt"] = (max(x_init, x), max(y_init, y))
        # img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
        
        
    def draw_rectangle(self, event, x, y, flags, params):
        global x_init, y_init, butt
        
        # First click initialize the init rectangle point
        if event == cv2.EVENT_LBUTTONDOWN:
            butt = "left"
            self.drawing = True
            x_init, y_init = x, y
            
        # Meanwhile mouse button is pressed, update diagonal rectangle point
        elif event == cv2.EVENT_MOUSEMOVE and self.drawing:
            self.update_pts(params, x, y)
            
        # Once mouse botton is release
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            self.update_pts(params, x, y)
            params["addroi"].append((x_init, y_init, x, y))
        
        # *****************************************************
        elif event == cv2.EVENT_RBUTTONDOWN:
            butt = "right"
            self.drawing = True
            x_init, y_init = x, y
            
        elif event == cv2.EVENT_RBUTTONUP:
            self.drawing = False
            self.update_pts(params, x, y)
            params.update({"removeroi" : (x_init, y_init, x, y)})

In [3]:
def main():
    pTime = 0
    cTime = 0
    
    event_params = {"side": "left", "top_left_pt": (-1, -1), "bottom_right_pt": (-1, -1), "addroi": [(-1.,-1.,-1.,-1)], "removeroi": (-1.,-1.,-1.,-1)}
        
    cap = cv2.VideoCapture(0)
    detector = HandAnchorCalculation()
    detector.drawing = False
    
    cv2.namedWindow('Webcam')
    # Bind draw_rectangle function to every mouse event
    cv2.setMouseCallback('Webcam', detector.draw_rectangle, event_params)

    while True:
        # Get image frame
        success, img = cap.read()
        
        # Find the hand and its landmarks with draw
        hands, img = detector.findHands(img)
        
        if hands:
            # first hand
            hand1 = hands[0]
            lmlist1 = hand1["lmlist"]  # List of 21 Landmark points
            bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
            centerPoint1 = hand1['center']  # center of the hand cx,cy
            handType1 = hand1["type"]  # Handtype Left or Right
            
            fingers1 = detector.findFingersUp(hand1, img)
            
            if len(hands) == 2:
                hand2 = hands[1]
                lmlist2 = hand2["lmlist"]  # List of 21 Landmark points
                bbox2 = hand2["bbox"]  # Bounding box info x,y,w,h
                centerPoint2 = hand2['center']  # center of the hand cx,cy
                handType2 = hand2["type"]  # Handtype Left or Right

                fingers2 = detector.findFingersUp(hand2, img)
                
                # Find Distance between two Landmarks. Could be same hand or different hands with draw
                length, info, img = detector.findHandDistance(lmlist1[8], lmlist2[8], img)
                
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
        
        # draw target
        # print(event_params["removeroi"][0])
        xrem0, yrem0, xrem0, yrem1 = event_params["removeroi"][0], event_params["removeroi"][1], event_params["removeroi"][2], event_params["removeroi"][3]
        for x in event_params["addroi"][1:]:
            xhist0, yhist0, xhist1, yhist1 = x[0], x[1], x[2], x[3]
            # print(x[0], x[1], x[2], x[3])
            cv2.rectangle(img, (xhist0, yhist0), (xhist1, yhist1), (255, 0, 255), 2)
            
            isremoved = not(max(xhist0, xhist1) < min(xrem0, xrem0) 
                         or min(xhist0, xhist1) > max(xrem0, xrem0)  
                         or max(yhist0, yhist1) < min(yrem0, yrem1) 
                         or min(yhist0, yhist1) < max(yrem0, yrem1))
            if isremoved:
                event_params["addroi"].remove(x)
            
            
        side, (x0,y0), (x1,y1) = event_params["side"], event_params["top_left_pt"], event_params["bottom_right_pt"]
        if side == "left":
            cv2.rectangle(img, (x0, y0), (x1, y1), (255, 0, 255), 2)
        else:
            cv2.rectangle(img, (x0, y0), (x1, y1), (255, 255, 255), 2)
        
        cv2.imshow("Webcam", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyWindow("Webcam")
    
if __name__ == "__main__":
    main()


error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
